In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import sklearn
from sklearn.model_selection import train_test_split

In [ ]:
# import data (just deal with level a for now)
data = pd.read_csv('OLIDv1.0/olid-training-v1.0.tsv', sep='\t', header=0, names=['id', 'tweet', 'subtask_a', 'subtask_b', 'subtask_c'])
data.head()

# Char-Level LSTM
## Preprocessing and Downsampling

In [ ]:
import re
import emoji
import itertools

def preprocess(tweet):
    # TODO: handles are comming up at @ user instead of @user, need to fix that
    # remove hashtags
    tweet = ' '.join(re.sub("(#[A-Za-z0-9]+)", " ", tweet).split())
    # remove non-ascii characters
    tweet = tweet.encode("ascii", "ignore").decode()
    # remove punctuation
    tweet = ' '.join(re.sub("[\.\,\!\?\:\;\-\=]", " ", tweet).split())
    # lowercase
    tweet = tweet.lower()
    # replace emoji with text rep
    tweet = emoji.demojize(tweet)
    tweet = tweet.replace(":"," ")
    tweet = ' '.join(tweet.split())
    # standardizing words
    tweet = ''.join(''.join(s)[:2] for _, s in itertools.groupby(tweet))
    return tweet

data.tweet = [preprocess(tweet) for tweet in data.tweet]

In [ ]:
print(data.tweet[922])

In [ ]:
off_df = data[data.subtask_a == 'OFF'].drop(['subtask_b', 'subtask_c'], axis=1)
not_df = data[data.subtask_a == 'NOT'].drop(['subtask_b', 'subtask_c'], axis=1)
not_df.tweet.head()

In [ ]:
# investigating class imbalance
import seaborn as sns
print(data.subtask_a.value_counts())
ax = sns.countplot(x='subtask_a', data=data, label='Offensive Language', palette='deep')
ax.set(xlabel='Tweets', ylabel='Counts')

In [ ]:
## downsampling
not_df = not_df.sample(n=len(off_df), random_state=12)
print(not_df.shape, off_df.shape)

In [ ]:
df = off_df.append(not_df).reset_index(drop=True)
sns.countplot(df.subtask_a)

In [ ]:
# get length column for each text
df['text_length'] = df.tweet.apply(len)

# get average char length by  label types
labels = df.groupby('subtask_a').mean()
labels

## Splitting into train/dev

In [ ]:
df['type'] = df['subtask_a'].map({'OFF': 1, 'NOT':0})
label = df.type.values

# split into train/dev
train, dev = sklearn.model_selection.train_test_split(df, test_size=0.1, random_state=0) 
dev = dev.reset_index(drop=True)
train = train.reset_index(drop=True)

train.head()

In [ ]:
from keras.utils import to_categorical

lstm_train_seqs = train.tweet
lstm_train_labs = train.type

lstm_train_labs = to_categorical(train.type)

lstm_dev_seqs = dev.tweet
lstm_dev_labs = dev.type

lstm_dev_labs = to_categorical(dev.type)

print(lstm_train_labs[:5])

## Model and Hyperparameter Definition 

In [ ]:
# Defining pre-processing hyperparameters
max_len = 50 # extended tweet char limit
trunc_type = "post" 
padding_type = "post" 
oov_tok = "<OOV>" 
vocab_size = 1000 # maximum number of unique tokens hence we can filter out rare words

## loading pretrained FastText Vectors

In [ ]:
from urllib.request import urlopen
import gzip

file = gzip.open(urlopen('https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ro.300.vec.gz'))
# file = open('wiki-news-300d-1M-subword.vec', 'r')
vectors = {}
for line in file:
    values = line.split()
    word = values[0].decode('utf-8')
    vector = np.array(values[1:], dtype='float32')
    vectors[word] = vector

vectors = load_vectors('wiki-news-300d-1M-subword.vec')
print(len(vectors.keys()))

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

word_tokenizer = Tokenizer(num_words = vocab_size, char_level=False, oov_token= oov_tok)
word_tokenizer.fit_on_texts(lstm_train_seqs)

In [ ]:
word_index = word_tokenizer.word_index
# check how many words 
tot_word = len(word_index)
print('There are %s unique words in training data. ' % tot_word) 

# Sequencing and padding on training and dev data
training_sequences = word_tokenizer.texts_to_sequences(lstm_train_seqs)
training_padded = pad_sequences(training_sequences, maxlen = max_len, padding = padding_type, truncating = trunc_type)

dev_sequences = word_tokenizer.texts_to_sequences(lstm_dev_seqs)
dev_padded = pad_sequences(dev_sequences, maxlen = max_len, padding = padding_type, truncating = trunc_type)

# Shape of train tensor
print('Shape of training tensor: ', training_padded.shape)
print('Shape of dev tensor: ', dev_padded.shape)

print(training_padded[0])

In [ ]:
# hyperparams
# vocab_size = 500 # As defined earlier
embedding_dim = 300
drop_value = 0.2 # dropout


In [ ]:
embedding_matrix = np.zeros((len(word_index)+1, embedding_dim))
for word, i in word_index.items():
    embedding_vector = vectors.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [ ]:
# Modeling 
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GlobalAveragePooling1D, Dense, Dropout, LSTM, Bidirectional, TimeDistributed

# LSTM model architecture
model = Sequential()
# char_model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(Embedding(len(word_index)+1, embedding_dim, input_length=max_len, weights=[embedding_matrix], trainable=False))
model.add(Bidirectional(LSTM(units=50, return_sequences=True, dropout=drop_value)))
# char_model.add(Dropout(0.5))
model.add(GlobalAveragePooling1D())
model.add(Dense(2, activation='softmax'))

model.summary()

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# fitting a dense spam detector model
num_epochs = 30
early_stop = EarlyStopping(monitor='val_loss', patience=5)
history2 = model.fit(training_padded, lstm_train_labs, epochs=num_epochs, validation_data=(dev_padded, lstm_dev_labs),\
                    callbacks =[early_stop], verbose=2)

In [ ]:
# this still exists
char_model.summary()

In [ ]:
test = pd.read_csv('OLIDv1.0/testset-levela.tsv', sep='\t', header=0, names=['id', 'tweet'])
test.tweet = [preprocess(tweet) for tweet in test.tweet]

# test.tweet = ruin(test)

test_sequences = word_tokenizer.texts_to_sequences(test.tweet)
test_padded = pad_sequences(test_sequences, maxlen = max_len, padding = padding_type, truncating = trunc_type)
print(test_padded.shape)
test_y = pd.read_csv('OLIDv1.0/labels-levela.csv', sep=',', header=None, names=['id', 'label'])
y_true = list(test_y.label.map({'OFF': 1, 'NOT':0}))


preds = model.predict(test_padded)#, test_y) 
# get the max
y_hat = [np.argmax(preds[i]) for i in range(len(preds))]


print(classification_report(y_true, y_hat))


# FastText Classifier

In [ ]:
# split into train/dev
# NOT DOWNSAMPLED
train, dev = sklearn.model_selection.train_test_split(data, test_size=0.1, random_state=0)
train.subtask_a.value_counts()

## Formatting data into FastText form and saving to CSV

In [ ]:
from nltk import word_tokenize

def get_fastText(data):
   fastTweet = [word_tokenize('__label__' + label + ' ' + tweet) for label, tweet in zip(data.subtask_a, data.tweet)]
   return fastTweet

import csv
  
def make_file(data, output_file, is_test=False):    
    with open(output_file, 'w') as csvoutfile:
        csv_writer = csv.writer(csvoutfile, delimiter=' ', lineterminator='\n')
        for row in get_fastText(data):
            csv_writer.writerow(row)

make_file(train, 'data/tweets.train')
make_file(dev, 'data/tweets.dev')

In [ ]:
## FastText classifier ##
import os
import fasttext

# reloading already trained model
# model = fasttext.load_model('models/fasttext.ftz')

def train_fasttext(train, dev):
    # ###################
    hyper_params = {"lr": 0.01,
                    "epoch": 100,
                    "wordNgrams": 2,
                    "dim": 20}     

    model = fasttext.train_supervised(input=os.path.join('data',train), **hyper_params)
    # reduces size of model and saves
    # model.quantize(input='tweets.train', qnorm=True, retrain=True, cutoff=100000)
    # model.save_model(os.path.join('models/','fasttext_downsampled' + ".ftz"))
    # ###################

    model_acc_training_set = model.test(os.path.join('data',train))
    model_acc_validation_set = model.test(os.path.join('data',dev))
    # DISPLAY ACCURACY OF TRAINED MODEL
    text_line = str(hyper_params) + " \naccuracy: " + str(model_acc_training_set[1])  + "\nvalidation: " + str(model_acc_validation_set[1]) + '\n' 
    print(text_line)
    return model



In [ ]:
from sklearn.metrics import classification_report
model_ft = train_fasttext('tweets.train', 'tweets.dev')

def get_preds(model, test_data):
    # involves some parsing to get just the letters 'OFF' or 'NOT
    return[model.predict(tweet)[0][0][-3:] for tweet in test_data.tweet]

def evaluate_fasttext(model, adv = False):
    test = pd.read_csv('OLIDv1.0/testset-levela.tsv', sep='\t', header=0, names=['id', 'tweet'])
    test.tweet = [preprocess(tweet) for tweet in test.tweet]
    if adv:
        test.tweet = ruin(test)
    
    y_hat = get_preds(model, test)
    
    test_y = pd.read_csv('OLIDv1.0/labels-levela.csv', sep=',', header=None, names=['id', 'label'])
    y_true = test_y.label
    print(classification_report(y_true, y_hat))
    
evaluate_fasttext(model_ft)


# Adversarial Examples

In [ ]:
from random import choice

def mispell(word):
    word = list(word)
    x = choice(range(4))

    if x == 0:
    # shuffle
        idx = choice(np.arange(len(word)-2))
        before = word[:idx]
        subset = word[idx:idx+2]
        after = word[idx+2:]
        subset = subset[::-1]
        word = before + subset + after
    if x == 1:
    # drop letters
        idx = choice(np.arange(len(word)))
        word = word[:idx] + word[idx+1:]
    if x == 2:
    # repeat a letter
        idx = choice(np.arange(len(word)))
        word = word[:idx] + [word[idx]] + word[idx:]
    if x == 3:
    # add period at random place
        idx = choice(np.arange(len(word)))
        word = word[:idx] + ['.'] + word[idx:]
    return ''.join(word)


In [ ]:
def ruin(data):
    adv = data.copy()
    NSFW = set(['fuck', 'damn', 'shit','dumbass', 'ass', \
                'bad', 'moron', 'idiot', 'mean', 'dumb', \
                'communist','terrible', 'cock', 'liberal', \
                'maga', 'democrat', 'conservative', 'trump', \
                'antifa', 'sick', 'toxic'])
    count = 0
    for ii, sent in enumerate(adv.tweet):
        words = sent.split()
        for jj, word in enumerate(words):
            # there will be some false positives, it's okay
            for bad_word in NSFW:
                if bad_word in word:
                    # also maybe add randomness in here, like a 50/50 chance it does permute
                    words[jj] = mispell(word)
                    new = ' '.join(words)
                    adv.tweet[ii] = new       
                    count += 1
    print(count) # 7492 subs
    return adv.tweet



In [ ]:
# train_adv, dev_adv = sklearn.model_selection.train_test_split(adv, test_size=0.1, random_state=0)
# train_adv.subtask_a.value_counts()

# make_file(train_adv, 'data/tweets.adv.train')
# make_file(dev_adv, 'data/tweets.adv.dev')

# model_adv = train_fasttext('tweets.adv.train', 'tweets.adv.dev')
# evaluate_fasttext(model_adv, adv=True)

## Testing Sentences for the chart

In [ ]:
print(model_ft.predict("@user someone should've taken this piece of shit to a volcano")) # 99.99%
print(model_ft.predict("@user someone should've taken this piece of sht to a volcano")) # 74.88 %

In [ ]:
print(model_ft.predict("@user liberaals are all kookoo")) # 85
print(model_ft.predict("@user liberals are all kookoo")) #98

In [ ]:
print(model_ft.predict("@user @user @user trump is a very sick man he is toxic and mean spirited")) # 99.6
print(model_ft.predict("@user @user @user trmup is a very siick man he is toxc and me.an spirited")) #98.5


In [ ]:
print(model_ft.predict('are you fucking serious url'))
print(model_ft.predict('are you not fucking serious url'))
#                     

In [ ]:
print(model_ft.predict('@user trump kicks dem butt its not so fun'))
print(model_ft.predict('@user trump kicks dem butt its so fun'))

In [ ]:
print(model_ft.predict('@user he is competing for worst president again'))
print(model_ft.predict('@user he is not competing for worst president again'))


In [ ]:
print(model_ft.predict('shit i will drown twitter in bullshhit if one of those is banned'))
print(model_ft.predict('shit i will not drown twitter in bullshhit if one of those is banned'))

